## Imports, function definitions

Note: DeepMoD does not work for python3.9

In [1]:
# pip install pysindy

In [2]:
# pip install DeePyMoD==2.1.0

In [3]:
# pip install --upgrade tensorflow

In [4]:
# pip install tensorflow-io

In [5]:
data_path = "datasets"
results_path = ""

In [6]:
from deepymod import DeepMoD

In [7]:
import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch
import pandas as pd

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold
from deepymod.training import train
from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pickle
def load_data(X,y):
    return torch.from_numpy(X).float(),torch.from_numpy(y).float()

def uxt_2D_to1D(u,x,t):
    '''
    Function for reshaping u,x and t
    '''
    t_reg=[]
    x_reg=[]
    for i in range(len(t)):
        for j in range(len(x)):
            t_reg.append(t[i])
            x_reg.append(x[j])
    u_reg = u.reshape((u.size, 1))

    return u_reg,x_reg,t_reg

descr = ['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']

def return_pde(w, rhs_description=descr, ut = 'u_t'):
    pde = ut + ' = '
    first = True
    for i in range(len(w)):
        if w[i] != 0:
            if not first:
                pde = pde + ' + '
            pde = pde + "(%05f)" % (w[i].real) + rhs_description[i] 
            first = False
    return pde


cuda


## Testing

In [8]:
# Simple PDE ut = 0.8ux

pred_coefficients=[]

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]
for j in range(len(snrs)):
    print("="*100)
    print("SNR = ",snrs[j])

    # Loading u,x,t
    path = data_path+"/4_"+str(j)+".pkl"
    file_to_read = open(path, "rb")
    data = pickle.load(file_to_read)

    # reshaping
    u_reg,x_reg,t_reg = uxt_2D_to1D(data["u"],data["x"],data["t"])
    X = np.vstack([t_reg,x_reg]).T
    y = u_reg

    # Deepmod
    preprocess_kwargs = {"noise_level": 0.0}
    load_kwargs = {"X": X, "y": y}
    dataset = Dataset(
    load_data,
    load_kwargs=load_kwargs,
    preprocess_kwargs=preprocess_kwargs,
    subsampler=Subsample_random,
    subsampler_kwargs={"number_of_samples": 5000},
    device=device,
    )
    train_dataloader, test_dataloader = get_train_test_loader(
    dataset, train_test_split=0.8
    )
    network = NN(2, [50, 50, 50,50], 1)
    library = Library1D(poly_order=2, diff_order=3) 
    estimator = Threshold(0.1) 
    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
    constraint = LeastSquares() 
    model = DeepMoD(network, library, estimator, constraint).to(device)
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
    train(
      model,
      train_dataloader,
      test_dataloader,
      optimizer,
      sparsity_scheduler,
      exp_ID="Test",
      write_iterations=25,
      max_iterations=20000,
      delta=1e-4,
      patience=200,
    )

    w = list(model.constraint_coeffs()[0])
    w = [w[i][0].item() for i in range(len(w))]
    print("\nPredicted PDE: ", return_pde(w))
    pred_coefficients.append([snrs[j]]+list(w))
    
df_4 = pd.DataFrame(pred_coefficients, columns = ["SNR"]+descr )
df_4 = df_4[['SNR', '', 'u_{x}', 'u_{xx}', 'u_{xxx}', 'u',
       'u^2', 'uu_{x}', 'u^2u_{x}', 'uu_{xx}', 'u^2u_{xx}', 'uu_{xxx}',
       'u^2u_{xxx}']]


SNR =  No noise
Dataset is using device:  cuda


2023-04-27 08:36:47.037447: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 08:36:47.554080: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 12950  MSE: 6.66e-04  Reg: 4.81e-06  L1: 9.97e-01 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (-0.794377)u_{x}
SNR =  20000
Dataset is using device:  cuda
 14575  MSE: 7.31e-04  Reg: 1.17e-05  L1: 9.92e-01 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (-0.809271)u_{x}
SNR =  15000
Dataset is using device:  cuda
  9250  MSE: 7.47e-04  Reg: 6.42e-06  L1: 9.95e-01 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (-0.794112)u_{x}
SNR =  10000
Dataset is using device:  cuda
 14000  MSE: 8.44e-04  Reg: 1.74e-05  L1: 1.02e+00 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (-0.789662)u_{x} + (-0.003269)u^2
SNR =  7500
Dataset is using device:  cuda
  6475  MSE: 9.12e-04  Reg: 1.47e-05  L1: 9.95e-01 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (-0.712145)u_{x} + (-0.166944)uu_{x}
SNR =  5000
Dataset is using device:  cuda
 11175  MSE: 9.71e-04  Reg: 1.38e-05  L1: 9.90e-01 Algorithm converged

PermissionError: [Errno 13] Permission denied: '/simplepde_results.csv'

In [9]:
df_4.to_csv(results_path+"simplepde_results.csv")
df_4

,SNR,,u_{x},u_{xx},u_{xxx},u,u^2,uu_{x},u^2u_{x},uu_{xx},u^2u_{xx},uu_{xxx},u^2u_{xxx}
0,No noise,0.0,-0.794377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20000,0.0,-0.809271,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,15000,0.0,-0.794112,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,10000,0.0,-0.789662,0.000000,0.000000,0.000000,-0.003269,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,7500,0.0,-0.712145,0.000000,0.000000,0.000000,0.000000,-0.166944,0.000000,0.000000,0.000000,0.000000,0.000000
5,5000,0.0,-0.793748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,3000,0.0,-0.795467,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,2000,0.0,-0.803259,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,1000,0.0,-0.800312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,800,0.0,-0.807226,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
# Heat equation ut = 0.8ux

pred_coefficients=[]

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

for j in range(len(snrs)):
    print("="*100)
    print("SNR = ",snrs[j])

    # Loading u,x,t
    path = data_path+"/1_"+str(j)+".pkl"
    file_to_read = open(path, "rb")
    data = pickle.load(file_to_read)

    # reshaping
    u_reg,x_reg,t_reg = uxt_2D_to1D(data["u"],data["x"],data["t"])
    X = np.vstack([t_reg,x_reg]).T
    y = u_reg

    # Deepmod
    preprocess_kwargs = {"noise_level": 0.0}
    load_kwargs = {"X": X, "y": y}
    dataset = Dataset(
    load_data,
    load_kwargs=load_kwargs,
    preprocess_kwargs=preprocess_kwargs,
    subsampler=Subsample_random,
    subsampler_kwargs={"number_of_samples": 5000},
    device=device,
    )
    train_dataloader, test_dataloader = get_train_test_loader(
    dataset, train_test_split=0.8
    )
    network = NN(2, [50, 50, 50,50], 1)
    library = Library1D(poly_order=2, diff_order=3) 
    estimator = Threshold(0.1) 
    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
    constraint = LeastSquares() 
    model = DeepMoD(network, library, estimator, constraint).to(device)
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
    train(
      model,
      train_dataloader,
      test_dataloader,
      optimizer,
      sparsity_scheduler,
      exp_ID="Test",
      write_iterations=25,
      max_iterations=20000,
      delta=1e-4,
      patience=200,
    )

    w = list(model.constraint_coeffs()[0])
    w = [w[i][0].item() for i in range(len(w))]
    print("\nPredicted PDE: ", return_pde(w))
    pred_coefficients.append([snrs[j]]+list(w))
    
df_1 = pd.DataFrame(pred_coefficients, columns = ["SNR"]+descr )
df_1 = df_1[['SNR', '', 'u_{x}', 'u_{xx}', 'u_{xxx}', 'u',
       'u^2', 'uu_{x}', 'u^2u_{x}', 'uu_{xx}', 'u^2u_{xx}', 'uu_{xxx}',
       'u^2u_{xxx}']]
df_1.to_csv(results_path+"heateqn_results.csv")
df_1

SNR =  No noise
Dataset is using device:  cuda
 19975  MSE: 3.24e+04  Reg: 5.17e-12  L1: 7.23e+06 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (0.000001) + (-1322336.125000)u_{xx} + (-114419.859375)u_{xxx} + (-3128.190674)uu_{x} + (1476.700928)uu_{xx} + (-2584.203857)uu_{xxx} + (19.397884)u^2u_{x} + (41.689983)u^2u_{xx} + (20.424377)u^2u_{xxx}
SNR =  20000
Dataset is using device:  cuda
 19975  MSE: 1.00e+05  Reg: 9.78e-13  L1: 5.12e+07 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (5.750172) + (292119.812500)u_{x} + (55139.574219)u_{xx} + (-11525.343750)u_{xxx} + (-0.487081)u + (16059.167969)uu_{x} + (4175.581055)uu_{xx} + (-1085.629639)uu_{xxx} + (0.009433)u^2 + (-1754.319092)u^2u_{x} + (-393.778229)u^2u_{xx} + (93.958221)u^2u_{xxx}
SNR =  15000
Dataset is using device:  cuda
 19975  MSE: 9.67e+04  Reg: 2.97e-15  L1: 7.64e+06 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (-443215.500000)u_{xx} + (-39198.375000)u_{xxx} + 

,SNR,,u_{x},u_{xx},u_{xxx},u,u^2,uu_{x},u^2u_{x},uu_{xx},u^2u_{xx},uu_{xxx},u^2u_{xxx}
0,No noise,6.227964e-07,0.000000e+00,-1.322336e+06,-1.144199e+05,0.000000e+00,0.000000e+00,-3128.190674,19.397884,1476.700928,41.689983,-2584.203857,20.424377
1,20000,5.750172e+00,2.921198e+05,5.513957e+04,-1.152534e+04,-4.870806e-01,9.432853e-03,16059.167969,-1754.319092,4175.581055,-393.778229,-1085.629639,93.958221
2,15000,0.000000e+00,0.000000e+00,-4.432155e+05,-3.919838e+04,0.000000e+00,0.000000e+00,-1291.824219,56.845459,-6109.179688,1127.136719,2684.224609,-42.213654
3,10000,2.062016e-01,4.653703e+05,1.450809e+05,-1.724602e+05,2.529142e-02,-1.780769e-03,-3280.689453,-1000.045593,6948.091797,-710.431641,-5683.994629,715.696838
4,7500,0.000000e+00,-4.766926e-01,0.000000e+00,7.102510e-02,-2.603607e-09,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,5000,-6.403378e-08,1.332130e+06,8.298172e+05,0.000000e+00,0.000000e+00,0.000000e+00,28507.621094,-5475.611816,-27350.873047,-961.510986,-2020.706055,109.722748
6,3000,1.435613e-08,0.000000e+00,-5.533544e-01,0.000000e+00,0.000000e+00,0.000000e+00,-2365.003906,24.304138,0.000000,0.000000,0.000000,0.000000
7,2000,1.170066e+00,6.396458e+05,2.466693e+05,-1.432913e+05,-7.289503e-03,-3.400256e-04,2695.617188,-322.464661,-59752.187500,1118.494873,-8079.178711,225.065414
8,1000,-7.155679e-02,6.663871e+05,9.162902e+04,-4.130628e+05,4.051483e-02,-4.181212e-04,-2076.195312,-51.854294,160.769531,-11.822596,-6405.975586,113.036819
9,800,7.349164e+00,2.889821e+05,3.910582e+04,-1.963331e+04,-9.855951e-02,2.726722e-04,4181.720703,-65.896660,-41.922241,-3.137567,256.043823,-0.659426


In [11]:
# Burgers eqn ut = -1uux +0.1uxx

pred_coefficients=[]

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

for j in range(len(snrs)):
    print("="*100)
    print("SNR = ",snrs[j])

    # Loading u,x,t
    path = data_path+"/2_"+str(j)+".pkl"
    file_to_read = open(path, "rb")
    data = pickle.load(file_to_read)

    # reshaping
    u_reg,x_reg,t_reg = uxt_2D_to1D(data["u"],data["x"],data["t"])
    X = np.vstack([t_reg,x_reg]).T
    y = u_reg

    # Deepmod
    preprocess_kwargs = {"noise_level": 0.0}
    load_kwargs = {"X": X, "y": y}
    dataset = Dataset(
    load_data,
    load_kwargs=load_kwargs,
    preprocess_kwargs=preprocess_kwargs,
    subsampler=Subsample_random,
    subsampler_kwargs={"number_of_samples": 5000},
    device=device,
    )
    train_dataloader, test_dataloader = get_train_test_loader(
    dataset, train_test_split=0.8
    )
    network = NN(2, [50, 50, 50,50], 1)
    library = Library1D(poly_order=2, diff_order=3) 
    estimator = Threshold(0.1) 
    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
    constraint = LeastSquares() 
    model = DeepMoD(network, library, estimator, constraint).to(device)
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
    train(
      model,
      train_dataloader,
      test_dataloader,
      optimizer,
      sparsity_scheduler,
      exp_ID="Test",
      write_iterations=25,
      max_iterations=20000,
      delta=1e-4,
      patience=200,
    )

    w = list(model.constraint_coeffs()[0])
    w = [w[i][0].item() for i in range(len(w))]
    print("\nPredicted PDE: ", return_pde(w))
    pred_coefficients.append([snrs[j]]+list(w))
    
df_2 = pd.DataFrame(pred_coefficients, columns = ["SNR"]+descr )
df_2 = df_2[['SNR', '', 'u_{x}', 'u_{xx}', 'u_{xxx}', 'u',
       'u^2', 'uu_{x}', 'u^2u_{x}', 'uu_{xx}', 'u^2u_{xx}', 'uu_{xxx}',
       'u^2u_{xxx}']]
df_2.to_csv(results_path+"burgerseqn_results.csv")
df_2

SNR =  No noise
Dataset is using device:  cuda
  6225  MSE: 2.76e-06  Reg: 6.87e-06  L1: 1.42e+00 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (0.098428)u_{xx} + (-0.992248)uu_{x}
SNR =  20000
Dataset is using device:  cuda
 19975  MSE: 5.43e-06  Reg: 6.48e-06  L1: 1.42e+00 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (0.099912)u_{xx} + (-0.991904)uu_{x}
SNR =  15000
Dataset is using device:  cuda
 19975  MSE: 5.08e-06  Reg: 4.30e-06  L1: 1.52e+00 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (0.098517)u_{xx} + (-0.995592)uu_{x}
SNR =  10000
Dataset is using device:  cuda
 13525  MSE: 4.44e-06  Reg: 3.07e-06  L1: 1.48e+00 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (0.099015)u_{xx} + (-0.995755)uu_{x}
SNR =  7500
Dataset is using device:  cuda
 16800  MSE: 7.75e-06  Reg: 7.52e-06  L1: 1.48e+00 Algorithm converged. Writing model to disk.

Predicted PDE:  u_t = (0.099128)u_{xx} + (-0.993309)uu_{x}
SNR

,SNR,,u_{x},u_{xx},u_{xxx},u,u^2,uu_{x},u^2u_{x},uu_{xx},u^2u_{xx},uu_{xxx},u^2u_{xxx}
0,No noise,0.0,0.000000,0.098428,0.000000,0.000000,0.000000,-0.992248,0.000000,0.000000,0.000000,0.000000,0.000000
1,20000,0.0,0.000000,0.099912,0.000000,0.000000,0.000000,-0.991904,0.000000,0.000000,0.000000,0.000000,0.000000
2,15000,0.0,0.000000,0.098517,0.000000,0.000000,0.000000,-0.995592,0.000000,0.000000,0.000000,0.000000,0.000000
3,10000,0.0,0.000000,0.099015,0.000000,0.000000,0.000000,-0.995755,0.000000,0.000000,0.000000,0.000000,0.000000
4,7500,0.0,0.000000,0.099128,0.000000,0.000000,0.000000,-0.993309,0.000000,0.000000,0.000000,0.000000,0.000000
5,5000,0.0,0.000000,0.099237,0.000000,0.000000,0.000000,-0.988658,0.000000,0.000000,0.000000,0.000000,0.000000
6,3000,0.0,0.000000,0.098146,0.000000,0.000000,0.000000,-0.989801,0.000000,0.000000,0.000000,0.000000,0.000000
7,2000,0.0,0.000000,0.098584,0.000000,0.000000,0.000000,-0.987220,0.000000,0.000000,0.000000,0.000000,0.000000
8,1000,0.0,0.000000,0.100100,0.000000,0.000000,0.000000,-0.984485,0.000000,0.000000,0.000000,0.000000,0.000000
9,800,0.0,0.000000,0.098812,0.000000,0.000000,0.000000,-0.986109,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
descr

['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']